In [43]:
!wget https://www.dropbox.com/s/tdp11r39faou5ae/cortex.zip?dl=0
!wget https://www.dropbox.com/s/o6nveqyk1k7f95z/expression_sparse.txt?dl=0

--2018-04-29 13:49:06--  https://www.dropbox.com/s/tdp11r39faou5ae/cortex.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/Aj5Z73aKMsoZ3Q0NCjMJBQW7H5baX5nurASBNcZBeplsc3EdE2xxUu0WCOeOWXJS/file [following]
--2018-04-29 13:49:07--  https://dl.dropboxusercontent.com/content_link/Aj5Z73aKMsoZ3Q0NCjMJBQW7H5baX5nurASBNcZBeplsc3EdE2xxUu0WCOeOWXJS/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.9.6, 2620:100:601f:6::a27d:906
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.9.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2556102 (2.4M) [application/zip]
Saving to: ‘cortex.zip?dl=0’

cortex.zip?dl=0     100%[===================>]   2.44M  --.-KB/s    in 0.1s    

2018-04-29 13:49:

In [0]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import scale

from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics

############################################################################################
# vae: build a variational autoencoder using the input data
# 
# This function is heavily derived from - 
# 
# Inputs:
#   - counts_mat: matrix with gene names as counts and individual cells are rows
#   - loss_fun: loss function for autoencoder
#   - latent_dim: number of latent dimensions to project data into
#   - intermediate_dim: number of dimensions in the hidden layer
#   - batch_size: batch size used for training autoencoder
#   - epochs: epochs for training autoencoder
#   - nbshape: shape parameter for negative binomial loss. Defaults to 1
# 
# Outputs:
#   - x_encoded: Encoded transformation of input
#   - z_mean: parameter for the mean of distribution in the latent space
#
############################################################################################

def vae(counts_mat, loss_fun, latent_dim, intermediate_dim, 
        batch_size, epochs, epsilon_std, nbshape=1):
            
    x_train = preprocess(counts_mat, loss_fun)
    original_dim = x_train.shape[1]
    num_samples = x_train.shape[0]
    
    # encoder network: map inputs to latent distribution parameters
    x = Input(shape=(original_dim,))
    h = Dense(intermediate_dim, activation='relu')(x)
    z_mean = Dense(latent_dim)(h)
    z_log_var = Dense(latent_dim)(h)

    
    # use 'sampling' function to sample new similar points from the latent space
    def sampling(args):
      z_mean, z_log_var = args
      epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.0, stddev=epsilon_std)
      return z_mean + K.exp(z_log_var / 2) * epsilon
    
    z = Lambda(sampling)([z_mean, z_log_var])
    
    # map these sampled latent points back to reconstructed inputs 
    # i.e. a sort of decoder network
    decoder_h = Dense(intermediate_dim, activation='relu')
    decoder_mean = Dense(original_dim, activation='sigmoid')
    h_decoded = decoder_h(z)
    x_decoded_mean = decoder_mean(h_decoded)


    # build end-to-end autoencoder
    vae_model = Model(x, x_decoded_mean)
    

    # build encoder, from inputs to latent space
    encoder = Model(x, z_mean)
    

    # Train the model using the end-to-end model, with a custom loss function: 
    # the sum of a reconstruction term, and the KL divergence regularization term.
    # Use different user defined loss functions to generate reconstruction loss
    
    if loss_fun == 'poisson':
      reconstruction_loss = original_dim*metrics.poisson(x, x_decoded_mean)
    elif loss_fun == 'negative_binomial':
      reconstruction_loss = original_dim*negative_binomial_loss(x, x_decoded_mean, nbshape)
    elif loss_fun == 'gaussian':
      reconstruction_loss = original_dim*metrics.mean_squared_error(x, x_decoded_mean)
    elif loss_fun == 'bernoulli':
      reconstruction_loss = original_dim*metrics.binary_crossentropy(x, x_decoded_mean)
    else:
      reconstruction_loss = original_dim*metrics.binary_crossentropy(x, x_decoded_mean) # defaults to cross entropy
        
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    
    vae_loss = K.mean(reconstruction_loss + kl_loss)    
    
    vae_model.add_loss(vae_loss)
    vae_model.compile(optimizer='rmsprop')
    vae_model.summary()
    
    
    # fit the model with the given data
    vae_model.fit(x_train, shuffle=True, epochs=epochs, batch_size=batch_size, validation_split=0.1)
    

    # project/encode inputs on the latent space
    encoder = Model(x, z_mean)
    x_encoded = encoder.predict(x_train, batch_size=batch_size)
    

    return x_encoded, z_mean

############################################################################################
# preprocess: preprocess counts matrix to make it suitable for particular loss function
#
# Inputs:
#   - Y: matrix with gene names as counts and individual cells are rows
#   - loss_fun: loss function for autoencoder
#
# Outputs:
#   - x_train: preprocessed matrix of counts for training the autoencoder
############################################################################################

def preprocess(counts_mat, loss_fun):
    if loss_fun == 'gaussian':
        x_train = np.log2(1 + (counts_mat.T/np.sum(counts_mat, axis=1)).T * 1e+6) # log2(1+Y/rowSums(Y)*1e6)
        x_train = scale(x_train, axis=1, with_mean=True, with_std=True, copy=True) # scale_center(Y)
        
    elif loss_fun == 'bernoulli':
        x_train = counts_mat
        x_train[x_train > 0] = 1 # int(Y>0)
        
    else:
        x_train = counts_mat
    
    return x_train

############################################################################################
# sampling: sample encoded values of the input using latent space parameters
#
# Inputs:
#   - args: mean and variance parameters of the latent space distribution
# 
# Outputs:
#   - encoded points sampled from the latent space distribution
############################################################################################


In [45]:
# unzip cortex data, move it all to one directory
!unzip cortex.zip?dl=0
!rm cortex.zip?dl=0
!mkdir cortex
!mv cell_info.csv cortex/
!mv cortex_expression.txt cortex/
!mv gene_info.csv cortex/

# move CBMC data to one directory
!mkdir CBMC_sparse
!mv expression_sparse.txt?dl=0 expression_sparse.txt
!mv expression_sparse.txt CBMC_sparse/

!ls

Archive:  cortex.zip?dl=0
  inflating: cell_info.csv           
  inflating: cortex_expression.txt   
  inflating: gene_info.csv           
mkdir: cannot create directory ‘cortex’: File exists
mkdir: cannot create directory ‘CBMC_sparse’: File exists
CBMC_sparse  cortex  datalab


In [48]:
cortex_counts = np.loadtxt('cortex/cortex_expression.txt')
cortex_counts = cortex_counts.T.astype('float32')
encoded_cortex_counts, cortex_counts_z_mean = vae(cortex_counts, loss_fun = 'gaussian', latent_dim = 50, intermediate_dim = 500, batch_size = 100, epochs = 100, epsilon_std = 1, nbshape=1)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Output "dense_78" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dense_78" during training.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 2000)         0                                            
__________________________________________________________________________________________________
dense_74 (Dense)                (None, 500)          1000500     input_16[0][0]                   
__________________________________________________________________________________________________
dense_75 (Dense)                (None, 50)           25050       dense_74[0][0]                   
__________________________________________________________________________________________________
dense_76 (Dense)                (None, 50)           25050       dense_74[0][0]                   
__________________________________________________________________________________________________
lambda_15 

2704/2704 [==============================] - 0s 134us/step - loss: 1344.7618 - val_loss: 1678.7485
Epoch 7/100
2704/2704 [==============================] - 0s 135us/step - loss: 1339.7140 - val_loss: 1673.4911
Epoch 8/100
2704/2704 [==============================] - 0s 136us/step - loss: 1336.2018 - val_loss: 1658.6051
Epoch 9/100
2704/2704 [==============================] - 0s 134us/step - loss: 1332.6018 - val_loss: 1671.1622
Epoch 10/100
2704/2704 [==============================] - 0s 134us/step - loss: 1330.4226 - val_loss: 1657.6675
Epoch 11/100
2704/2704 [==============================] - 0s 128us/step - loss: 1327.7597 - val_loss: 1660.1555
Epoch 12/100
2704/2704 [==============================] - 0s 130us/step - loss: 1326.6340 - val_loss: 1655.4496
Epoch 13/100
2704/2704 [==============================] - 0s 132us/step - loss: 1324.7642 - val_loss: 1652.5421
Epoch 14/100
2704/2704 [==============================] - 0s 136us/step - loss: 1324.0218 - val_loss: 1662.0865
Epoch 15

2704/2704 [==============================] - 0s 134us/step - loss: 1316.3547 - val_loss: 1648.8987
Epoch 22/100
2704/2704 [==============================] - 0s 131us/step - loss: 1315.3806 - val_loss: 1650.2462
Epoch 23/100
2704/2704 [==============================] - 0s 131us/step - loss: 1314.3546 - val_loss: 1669.4638
Epoch 24/100
2704/2704 [==============================] - 0s 131us/step - loss: 1313.9227 - val_loss: 1636.3868
Epoch 25/100
2704/2704 [==============================] - 0s 132us/step - loss: 1312.5825 - val_loss: 1635.0308
Epoch 26/100
2704/2704 [==============================] - 0s 132us/step - loss: 1312.1751 - val_loss: 1630.9414
Epoch 27/100
2704/2704 [==============================] - 0s 132us/step - loss: 1310.8565 - val_loss: 1627.6214
Epoch 28/100
2704/2704 [==============================] - 0s 133us/step - loss: 1310.1658 - val_loss: 1627.7738
Epoch 29/100
2704/2704 [==============================] - 0s 138us/step - loss: 1309.6444 - val_loss: 1636.1333
Epoch

2704/2704 [==============================] - 0s 132us/step - loss: 1304.9166 - val_loss: 1629.5682
Epoch 37/100
2704/2704 [==============================] - 0s 133us/step - loss: 1305.0530 - val_loss: 1629.0922
Epoch 38/100
2704/2704 [==============================] - 0s 128us/step - loss: 1304.1069 - val_loss: 1606.6340
Epoch 39/100
2704/2704 [==============================] - 0s 129us/step - loss: 1303.5082 - val_loss: 1602.9646
Epoch 40/100
2704/2704 [==============================] - 0s 132us/step - loss: 1303.0491 - val_loss: 1617.6061
Epoch 41/100
2704/2704 [==============================] - 0s 127us/step - loss: 1302.3341 - val_loss: 1602.1371
Epoch 42/100
2704/2704 [==============================] - 0s 126us/step - loss: 1301.7397 - val_loss: 1605.1137
Epoch 43/100
2704/2704 [==============================] - 0s 131us/step - loss: 1301.3303 - val_loss: 1601.0110
Epoch 44/100
2704/2704 [==============================] - 0s 135us/step - loss: 1300.9107 - val_loss: 1607.5635
Epoch

2704/2704 [==============================] - 0s 132us/step - loss: 1297.0471 - val_loss: 1596.4159
Epoch 52/100
2704/2704 [==============================] - 0s 139us/step - loss: 1297.2050 - val_loss: 1586.8859
Epoch 53/100
2704/2704 [==============================] - 0s 129us/step - loss: 1296.5483 - val_loss: 1587.8474
Epoch 54/100
2704/2704 [==============================] - 0s 128us/step - loss: 1296.6898 - val_loss: 1589.8949
Epoch 55/100
2704/2704 [==============================] - 0s 129us/step - loss: 1295.6949 - val_loss: 1587.3476
Epoch 56/100
2704/2704 [==============================] - 0s 132us/step - loss: 1295.4613 - val_loss: 1584.5341
Epoch 57/100
2704/2704 [==============================] - 0s 134us/step - loss: 1294.7039 - val_loss: 1591.3459
Epoch 58/100
2704/2704 [==============================] - 0s 137us/step - loss: 1294.4726 - val_loss: 1586.7800
Epoch 59/100
2704/2704 [==============================] - 0s 135us/step - loss: 1293.9055 - val_loss: 1584.6343
Epoch

2704/2704 [==============================] - 0s 145us/step - loss: 1291.6819 - val_loss: 1577.4060
Epoch 67/100
2704/2704 [==============================] - 0s 128us/step - loss: 1291.1956 - val_loss: 1582.5473
Epoch 68/100
2704/2704 [==============================] - 0s 130us/step - loss: 1290.5235 - val_loss: 1581.0298
Epoch 69/100
2704/2704 [==============================] - 0s 129us/step - loss: 1290.5948 - val_loss: 1591.1243
Epoch 70/100
2704/2704 [==============================] - 0s 129us/step - loss: 1290.1640 - val_loss: 1596.3091
Epoch 71/100
2704/2704 [==============================] - 0s 129us/step - loss: 1289.7990 - val_loss: 1588.3214
Epoch 72/100
2704/2704 [==============================] - 0s 131us/step - loss: 1289.5046 - val_loss: 1576.4429
Epoch 73/100
2704/2704 [==============================] - 0s 148us/step - loss: 1289.3289 - val_loss: 1583.8658
Epoch 74/100
2704/2704 [==============================] - 0s 136us/step - loss: 1288.9663 - val_loss: 1580.7954
Epoch

2704/2704 [==============================] - 0s 132us/step - loss: 1286.8162 - val_loss: 1580.4022
Epoch 82/100
2704/2704 [==============================] - 0s 126us/step - loss: 1286.4660 - val_loss: 1571.6062
Epoch 83/100
2704/2704 [==============================] - 0s 130us/step - loss: 1285.9305 - val_loss: 1571.7184
Epoch 84/100
2704/2704 [==============================] - 0s 129us/step - loss: 1286.1777 - val_loss: 1569.3128
Epoch 85/100
2704/2704 [==============================] - 0s 126us/step - loss: 1285.1772 - val_loss: 1575.4701
Epoch 86/100
2704/2704 [==============================] - 0s 129us/step - loss: 1285.1383 - val_loss: 1572.2560
Epoch 87/100
2704/2704 [==============================] - 0s 132us/step - loss: 1284.8775 - val_loss: 1577.9786
Epoch 88/100
2704/2704 [==============================] - 0s 132us/step - loss: 1284.4710 - val_loss: 1587.1829
Epoch 89/100
2704/2704 [==============================] - 0s 132us/step - loss: 1284.3145 - val_loss: 1591.1600
Epoch

2704/2704 [==============================] - 0s 129us/step - loss: 1282.2582 - val_loss: 1574.8333
Epoch 97/100
2704/2704 [==============================] - 0s 129us/step - loss: 1282.1879 - val_loss: 1578.9421
Epoch 98/100
2704/2704 [==============================] - 0s 135us/step - loss: 1281.5851 - val_loss: 1570.7933
Epoch 99/100
2704/2704 [==============================] - 0s 136us/step - loss: 1281.9055 - val_loss: 1576.5374
Epoch 100/100
2704/2704 [==============================] - 0s 133us/step - loss: 1281.6269 - val_loss: 1575.4139
